# ECE 227: Homework 3
## Part (A): Network Visualization

In [ ]:
%%time
#import useful packages, all of them are important but not necessarily used in this code
#enable inline plotting in Python Notebook
#supress warnings

%pylab inline
import networkx as nx
import numpy as np
import matplotlib
import scipy
import warnings
warnings.filterwarnings('ignore')
import time
import os

### You can access the network datasets exactly in the way you accessed them for HW 1. 
### Please DO NOT copy the datasets to your directory because it could result in 'timeout' errors during the submission. 
### Now that you are familiar with Gephi and its usage, you will explore some of its built-in tools to improve your visualizations. You will need your results from HW1. Only use the results obtained using NetworkX here. This can be done by adding your results on the node degrees, centralities etc. as attributes to the nodes.

### In this task, we will use the Filter tool of Gephi to threshold the available network data using various properties.
### Visualise the Facebook, Enron-emails and Collaboration (Erdos) networks while applying the following thresholds. Make sure to have all the visualizations labelled with appropriate node labels. This is quite an open-ended question, as you have a lot of scope to make your visualizations better by trying out different layouts, colors etc. So, turn in the best visualization that you get in each case. You should attach an image (.png, .jpg) for each visualization here in the notebook itself. Also, make sure that it is well readable.
### (1) Top ~20% nodes, thrsholded by Node Degree
### (2) Top ~1% nodes, thrsholded by Node Degree
### (3) Top ~20% nodes, thrsholded by Betweeness Centrality
### (4) Top ~1% nodes, thrsholded by Betweeness Centrality

### You may reproduce some of your HW 1 images in response to this task.

"""Facebook top 20 percent degree"""
![Facebook_top20_degree](facebook_20node.png)
"""Facebook top 1 percent degree"""
![Facebook_top1_degree](facebook_node1.png)
"""Facebook top 20 percent betweeness"""
![Facebook_top20_between](facebook_20between.png)
"""Facebook top 1 percent betweeness"""
![Facebook_top1_between](facebook_between1.png)
"""Enron top 20 percent degree"""
![enron_top20perc_degree](enron_degree20.png)
"""Enron top 1 percent degree"""
![enron_top1perc_degree](enron_degree1perc.png)
"""Enron top 20 percent betweeness"""
![Enron_top20_between](enron_between20.png)
"""Enron top 1 percent betweeness"""
![Enron_top1_between](enron_betweentop1.png)
"""Erdos top 20 percent degree"""
![Erdos_top20percdeg](erdos_degree20.png)
"""Erdos top 1 percent degree"""
![Erdos_top20percdeg](erdos_degree1.png)
"""Erdos top 20 percent betweeness"""
![Erdos_top20percdeg](erdos_between20.png)
"""Erdos top 1 percent betweeness"""
![Erdos_top20percdeg](erdos_betweentop1.png)

## Part (B): Community Detection

### In this task, we will try to find communities in the given networks and learn more about them. There are libraries that can be used with NetworkX for community detection (http://perso.crans.org/aynaud/communities/). In addition to NetworkX, we will also use igraph library in this task for community detection purposes. If not already available, install the following required packages and have a look at their documentations to gain some familiarity with them. More information on community detection can also be found here: https://arxiv.org/abs/0906.0612

In [ ]:
#!pip install python-louvain
#!pip install python-igraph

### There are multiple algorithms to detect communities. One of the commonly used algorithms is Louvain algorithm. The method is a greedy optimization method that attempts to optimize the "modularity" of a partition of the network. The 'community' library uses Louvain algorithm, and hence we get partitions based on optimized modularity. Implement a python code using the 'community' library to find communities in the Citations network and the Collaboration network (Erdos). Write your code in the next cell and visualize your community detection results in Gephi for both the networks. Label the nodes in the visualization properly. Use the largest connected components, if required. Include the images (.jpg, .png) of your visualizations here.

In [ ]:
import community
import urllib

try:
    os.makedirs("../data")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
try:
    os.makedirs("../data/citnet")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
        
urllib.request.urlretrieve("http://snap.stanford.edu/data/cit-HepTh.txt.gz","../data/citnet/citnet_combined.txt.gz")

import gzip
# datapath = "../../../datasets/ece227-fa19-public/"
inF = gzip.GzipFile("../data/citnet/citnet_combined.txt.gz", 'rb')
s = inF.read()
inF.close()

outF = open("../data/citnet/citnet_combined.txt", 'wb')
outF.write(s)
outF.close()

file_name = "../data/citnet/citnet_combined.txt"
#convert the information in the text file into a graph, find no. of edges & nodes in the graph

g3=nx.read_edgelist(file_name,create_using=nx.Graph(),nodetype=int)
node, edge=g3.order(),g3.size()
print(nx.info(g3))

def deg_dict_generator(G):
    dic = {}
    for node in G.nodes():
        dic[node] = G.degree(node)
    
    return dic


deg_dictionary3 = deg_dict_generator(g3)

nx.set_node_attributes(g3,deg_dictionary3,'degree')


partition = community.best_partition(g3,deg_dictionary3)

nx.set_node_attributes(g3,partition,'community')
nx.write_gml(g3,'citnet.gml')



try:
    os.makedirs("data/erdos")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
        
# IF THE FOLLOWING FAILS DOWNLOAD THE PAGE MANUALLY...

#urllib.request.urlretrieve("https://files.oakland.edu/users/grossman/enp/Erdos1.html", "data/erdos/Erdos1.html") 

g4 = nx.Graph()
dict_authors = {}
dict_authors['Paul Erdos'] = 0
g4.add_node(0)
g4.nodes[0]['author'] = 'Paul Erdos'

# add the authors with Erdos number 1 and 2 from file
line_count = 1
skip_line = 24
skip_space = 1

is_new = False
author = ""
coauthor = ""
index = 1
ind_author = 1
ind_coauthor = 1

def parseLine(l, start):
    end = start
    while end < len(l) - 1 and not (l[end] == ' ' and l[end + 1] == ' '):
        end += 1
    return l[start:end]

def addAuthor(auth, ind):
    if auth in dict_authors:
        return ind
    dict_authors[auth] = ind
    return ind + 1

for l in open("C:/Users/brand/Desktop/Erdos1.html"):    
    if line_count >= skip_line:
        if l == '\n':
            is_new = True
        elif is_new:
            author = parseLine(l, 0)
            index = addAuthor(author, index)
            ind_author = dict_authors[author]
            g4.add_edge(0, ind_author)
            g4.node[ind_author]['author'] = author
            is_new = False
        elif l == '</pre>':
            break
        else:
            coauthor = parseLine(l, skip_space)
            index = addAuthor(coauthor, index)
            ind_coauthor = dict_authors[coauthor]
            g4.add_edge(ind_author, ind_coauthor)
            g4.node[ind_coauthor]['author'] = coauthor
    line_count += 1

print(nx.info(g4))

def deg_dict_generator(G):
""""""    dic = {}
    for node in G.nodes():
        dic[node] = G.degree(node)
    
    return dic


deg_dictionary4 = deg_dict_generator(g4)

nx.set_node_attributes(g4,deg_dictionary4,'degree')


partition = community.best_partition(g4,deg_dictionary4)

nx.set_node_attributes(g4,partition,'community')
nx.write_gml(g4,'erdosnethw2.gml')


"""Citations Network top 3 Communites using Louvain"""

![Citations Network Community top 3](citationnetwork_community_top3_COMMUNITY.png)

"""Erdos Network top 3 Communites using Louvain"""
![Erdos Network Community top 3](erdosnetwork_top4communities_COMMUNITY.png)


### Compared to 'community' library, 'igraph' has more flexibilty to detect communities. Igraph allows the user to partition the network into the number of communities that the user wishes. Obviously this number is bounded. Now, you will use this feature to divide the given network into '5' communities using 'igraph' and observe the results. Write a python code to implement the above task for the Citation and the Erdos networks. Remember that unlike 'community', igraph provides multiple approaches to community detection, the most obvious approach being the greedy one because it optimizes modularity. Visualize your community detection results in Gephi for both the networks. Label the nodes in the visualization properly. Use largest connected components if required. Use different colors for nodes in every community. Include the images(.jpg, .png) of your visualizations here.

In [ ]:
import igraph
g = igraph.Graph.from_networkx(g3)
GG = g.clusters().giant()
com = GG.community_leading_eigenvector(clusters=5)

membership = com.membership
GG.vs["membership"] = membership

GG.write_gml('citationnet.gml')


G = igraph.Graph.from_networkx(g4)
G.delete_vertices(1)
comG = G.community_leading_eigenvector(clusters=5)

membershipG = comG.membership
G.vs["membership"] = membershipG

degs = G.degree()
G.vs['degree'] = degs

G.write_gml('erdosnetigraph.gml')

"""Top 5 citation net communities using Igraph"""
![Citations Network Igraph top 5](citationnet_igraph_5communities.png)

"""Top 5 Erdos net communities using Igraph"""
![Erdos Network Igraph top 5](erdosnetigraph_5communitesreal_IGRAPH.png)

### Now that you have detected the communities, you will analyze your results further. This task is only for the Collaboration network (Erdos). Use results from the communtiy detection performed using 'community'. Sort the communities and get the largest 5 communities. For each of these 5 communities, get 3 nodes with the highest node degree. So you will get 3 authors per community, for 5 communities. Now, find out the areas of research of each of these authors and enlist them. Further, observe if there is any reason for these 3 authors to be in same community (do this for each community). State this reason in brief. Write all of your results in the next cell. Also include any other interesting results that you may observe during the process.

In [ ]:
"""
Community 1:
    
    Paul Erdos- Mathematics
    Ralph Philip Boas Jr- Mathematics/ complex analysis
    Samuel James Taylor- MAthematics/Brownian motion and Fractals 
    
    Each author is active in abstract non-applied mathematics. 

Community 2-
    Noga Alon- Mathematics/combinatorics and graph theory
    Zsolt Tuza- Mathematics/graph theory
    Douglas Brent- Graph theory
    
    The three authors are all involved in graph theory and combinatorics.

Community 3-
    Frank Harary-Graph theory
    Pavol Hell-Math and computer science/algorithms and computational combinatorics 
    Douglas Robert Stinson- Math/cryptography 
    
    Again, the authors share a common interest in the application of mathematics to computer science.

Community 4-
    Stephen Travis Hedentiemi- math and computer science/ graphs and geneology
    Michael Anthony Henning- math/geneology 
    Wayne Dean Goddard- Geneology and graph theory
   
   Here the relevant authors are involved in geneology and graph theory 
    
Community 5-
    Charles Joseph Colbourn- cs and math focusing on graph algorithms and combinational design
    Saharon Shelah- Mathematical Logic
    Peter Salamon- Thermodynamics,optimization and biomathematics 
    
    This is the one community where there is some slight heterogeneity among the author's areas of research, but they are still focused primarily on applied mathematics. 

It is clear that the individuals in each community study a similar field of mathematics. The papers published in the five communities demonstrate a homogeneity of focus. The one exception could be group 1, but the three ares of focus among the three authors is still pure nonapplied mathematics.

"""

## Part (C) COVID19 Simulations

### In this homework we are going to gain some basic familiarity with a Susceptible-Exposed-Infectious-Recovered/deceased (SEIR)-based simulation model for estimating the number of infections, hospitalizations, and deaths due to COVID19. The model that we are going to use is a version of SEIR model used for the https://covid19-projections.com/ that was one of the very successful projection websites for COVID19. You can download (clone) the simulator from here https://github.com/youyanggu/yyg-seir-simulator and upload the *necessary* files to the datahub (do not upload the entire source code and parameters and try to put them outside your submission folder to avoid submission error).
### Using this simulator answer the following questions:

### 1. In one paragraph explain how the SEIR model works and find and explain the part of the library that has to do with the core idea of the SEIR model that calculates the number of deaths (the second part has extra points).

In [ ]:
"""
The SEIR model categorizes a population's individuals into one of four groups: susceptible,exposed,infected or recovered. At any
time, an individual may move from one of these four states to another. The likelihood that an individual moves from susceptible
to infected depends on paramters like the contraction rate, social distancing measures, and other environemntal factors.
Similarly, the probability an infected individual succumbs to the illness depends on the mortality rate of the virus. The
model used by Yang seeks to learn these parameters by grid search, or by running the model on the observed data under various 
hyperparameters and giving higher weight to the parameters that minimize the loss function better. These parameters are learned
by minimizing a loss function on the JHU daily death dataset. Once these paramaters are learned, the parameters can be fed into 
the SEIR model, which simulates how the virus spreads and the rate at which the population succumbs to the virus based on the 
learned parameters. 

"""

### 2. Using this simulation model whose parameters are last updated in October 2020, predict the total number of deaths by the end of 2020 and compare the result with the actual number of deaths in the US (you can use John Hopkins website for the reference: https://coronavirus.jhu.edu/us-map). Is the model over-estimating or under-estimating the number of deaths? In your opinion why the model predicts the number of deaths differently?

In [ ]:
"""
Estimated deaths ending on 12/31/2020:        328,274 deaths
Actual deaths on 12/31/2020 according to CDC: 342,199 deaths

The model under-estimates the death total but only by 4 percent. I am not qualified to weigh in on the matter, but I would 
propose the model does not properly adjust for the seasonality of the virus in the winter months, or underestimated the 
lockdown's fatigue effects and the population's growing distaste of lockdown and social distancing measures. 
"""

### 3. If the initial R_0 of the model was increased by 20% how many more deaths would have occured compared to the model's original prediction in the US? 

In [ ]:
"""
Adjusting R_0 to 2.46 from 2.26 leads to:

Estimated deaths under new R0 - 310,185
Original Estimated deaths - 224,263

Difference of 85,922 deaths 
"""

### 4. Answer the above question for the case where the openning date in the US was increased by 10 days and decreased by 10 days.

In [ ]:
"""
Changing the reopening date to 4/30/20 and 5/20/2020 from 5/10/2020:

Estimated deaths with 4/30 opening date- 245,311
Estimated deaths with 5/10 opening date- 224,263
Estimated deaths with 5/20 opening date- 205,355

"""

### 5. According to this model, how many fewer deaths would have occurred if the social distancing in the US had started just 2 days earlier? How about 4, 7, and 10 days?

In [ ]:
"""

2 days-177,802 total estimated deaths ---> 46,461 estimated fewer deaths
4 days- 138,615 total estimated deaths ---> 85,648 estimated fewer deaths
7 days- 92,880 total estimated deaths ---> 131,383 estimated fewer deaths
10 days- 61,091 total estimated deaths ---> 163,172 estimated fewer deaths

I am subtracting the models estimate under the best parameters, not the actual death total, from the new social
distancing dates

"""